## Data

In [1]:
import numpy as np
import tensorflow as tf

from hopfield import DenseRecon, DiscreteTimeHopfieldLayer, sign
from mnist import load_mnist

tf.random.set_seed(42)

tf.keras.backend.clear_session()

print(tf.__version__)

2.3.0


In [2]:
IMAGE_SIZE = (32, 32)
# IMAGE_SIZE = (8, 8)  # XXX: test!
BINARIZE = True
# BINARIZE = False

In [3]:
(x_train, _), _ = load_mnist(image_size=IMAGE_SIZE, binarize=BINARIZE)

## Model

In [4]:
def softly_sign(x, from_logits=False):
    """Returns 1 if x > threshold else -1, element-wisely, with the gradients
    :math:`\partial f_i / \partial x_j = \delta_{i j}`, i.e. an unit Jacobian.

    Parameters
    ----------
    x : tensor
    from_logits : bool, optional
        If true, then softly binarize sigmoid(x) instead of x.

    Returns
    -------
    tensor
        The same shape and dtype as x.
    """

    def identity(dy):
        return dy

    @tf.custom_gradient
    def fn(x):
        y = sign(x)
        return y, identity

    return fn(tf.nn.tanh(x)) if from_logits else fn(x)

In [5]:
def create_model(num_repeat):
    model = tf.keras.Sequential([
        tf.keras.layers.RepeatVector(num_repeat),
        tf.keras.layers.Flatten(),
        DiscreteTimeHopfieldLayer(
            DenseRecon(
                activation=tf.tanh,
                binarize=sign,
            ),
            max_steps=100,
            reg_factor=1),
        tf.keras.layers.Reshape([num_repeat, -1]),
        tf.keras.layers.Lambda(
            lambda x: softly_sign(tf.reduce_mean(x, axis=-2))),
    ])
    model.compile(optimizer='adam')
    return model

In [6]:
model = create_model()
X = x_train[:100].numpy()

import pickle
with open('../dat/training_z.pkl', 'rb') as f:
    training_z = pickle.load(f)
training_z = training_z * 2 - 1
X = training_z[:100]

ds0 = tf.data.Dataset.from_tensor_slices(X)
ds = ds0.shuffle(10000).repeat(10000).batch(128)
model.fit(ds)

7813/7813 [==============================] - 43s 5ms/step - loss: 0.0286


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
repeat_vector (RepeatVector) (None, 8, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
discrete_time_hopfield_layer (None, 512)               262657    
_________________________________________________________________
reshape (Reshape)            (None, 8, 64)             0         
_________________________________________________________________
lambda (Lambda)              (None, 64)                0         
Total params: 262,657
Trainable params: 262,656
Non-trainable params: 1
_________________________________________________________________


In [8]:
# noised_X = X + np.random.normal(size=X.shape) * 0.3
noised_X = np.where(np.random.random(size=X.shape) < 0.3, -X, X)
recon_X = model.predict(noised_X)

print('Relax steps:', model.layers[2].final_step.numpy())

orig_err = noised_X - X
err = recon_X - X
print(f'{np.quantile(np.abs(orig_err), 0.99)} => '
      f'{np.quantile(np.abs(err), 0.99)}')

Relax steps: 99
2.0 => 2.0


In [9]:
X, recon_X

(<tf.Tensor: shape=(100, 64), dtype=float32, numpy=
 array([[-1.,  1., -1., ..., -1., -1.,  1.],
        [-1., -1., -1., ..., -1., -1.,  1.],
        [ 1.,  1., -1., ..., -1.,  1., -1.],
        ...,
        [-1.,  1.,  1., ...,  1., -1.,  1.],
        [ 1.,  1., -1., ..., -1., -1., -1.],
        [-1.,  1.,  1., ...,  1., -1., -1.]], dtype=float32)>,
 array([[ 1.,  1., -1., ..., -1.,  1.,  1.],
        [ 1., -1., -1., ..., -1.,  1.,  1.],
        [ 1., -1.,  1., ..., -1.,  1., -1.],
        ...,
        [-1.,  1., -1., ...,  1., -1.,  1.],
        [-1.,  1., -1., ..., -1., -1.,  1.],
        [-1.,  1., -1., ...,  1., -1., -1.]], dtype=float32))

## Conclusions and Discussions